In [1]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
df_for_dimension_class = pd.read_csv("radar_dataset/df_for_dimension_class.csv")

data_labels_for_training = []
for i in range(1, 201):
        data_labels_for_training.append("np" + str(i))
data_labels_for_training.append("view_angle")
data_labels_for_training.append("distance_to_radar")

target_label = "dimension_class"

data = df_for_dimension_class[data_labels_for_training].to_numpy()
labels = df_for_dimension_class[target_label].to_numpy()

In [3]:
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, train_size=0.8)

pool = Pool(data, labels)
pool_train = Pool(train_data, train_labels)
pool_test = Pool(test_data, test_labels)

model = CatBoostClassifier(iterations=1500,
                           depth=3,
                           learning_rate=0.2,
                           loss_function='MultiClass',
                           task_type="GPU",
                           early_stopping_rounds = 20,
                           verbose=False)

In [4]:
model.fit(pool_train, eval_set=pool_test, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [5]:
preds_class_test = model.predict(pool_test)
preds_class_test = preds_class_test.transpose()
print(model.score(test_data, test_labels))

0.9064089290387175


In [6]:
preds_class_train = model.predict(pool_train)
preds_class_train = preds_class_train.transpose()
print(model.score(train_data, train_labels))

0.9135364223341014


In [7]:
from catboost import cv

In [8]:
params = {}
params['loss_function'] = 'MultiClass'
params['iterations'] = 1500
params['custom_loss'] = 'AUC'
params['learning_rate'] = 0.2
params['task_type'] = 'GPU'
params['depth'] = 3
params['early_stopping_rounds'] = 20

In [9]:
cv_data1 = cv(
    params = params,
    pool = pool,
    fold_count = 5,
    shuffle = True,
    partition_random_seed = 0,
    plot=True,
    stratified = False,
    verbose = False
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/5]


AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


bestTest = 0.2753853363
bestIteration = 1499
Training on fold [1/5]


AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


...

bestTest = 0.2736732936
bestIteration = 1499
Training on fold [3/5]


AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


bestTest = 0.2748629221
bestIteration = 1499
Training on fold [4/5]


AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


bestTest = 0.2796655137
bestIteration = 1499


На кросс-валидации проверили что качество не сильно зависит от разбиения